In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import GPy

%load_ext autoreload
%autoreload 2

TODO: Changed name of files for sensor1 and also placing sensor_ref in a csv file now. Need to update the code

# Sensor Model Tip Curve

In [ ]:
data_tbl = pd.read_csv('../data/all_datasets/calibration/sensor1_tip_calib_May_04.csv')

In [ ]:
atten = 50
pos_ref = np.array([[64.30451325,98.0687005,16.1023935]]) # may after new sensor stand

P = data_tbl[['px','py','pz']].to_numpy() - pos_ref
S = data_tbl[['sx','sy']].to_numpy()/atten

In [ ]:
ker1 = GPy.kern.ThinPlate(3, variance=1, R=100)
mx = GPy.models.GPRegression(P,S[:,0:1],ker1)
my = GPy.models.GPRegression(P,S[:,1:2],ker1)

from sklearn.metrics import r2_score

y_pred = mx.predict(P)[0]
r2_x = r2_score(y_true = S[:,0:1], y_pred = y_pred )
print(f'Sensor model X R2 score {r2_x}')
y_pred = my.predict(P)[0]
r2_y = r2_score(y_true = S[:,1:2], y_pred = y_pred )
print(f'Sensor model Y R2 score {r2_y}')

In [ ]:
import plotly.graph_objects as go
from sklearn.metrics import r2_score

# get points that are near the zero plane
select_zero_idx = np.where(np.abs(P[:,2]) < 0.01)[0]

y_pred = mx.predict(P)[0]
r2_score = r2_score(y_true = S[:,0:1], y_pred = y_pred )
print(f'Sensor model R2 score {r2_score}')

margin = 20
x_range = np.arange(min(P[:,0])-margin, max(P[:,0])+margin)
y_range = np.arange(min(P[:,1])-margin, max(P[:,1])+margin)
[X_mesh, Y_mesh] = np.meshgrid(x_range, y_range)
Z_mesh = np.zeros_like(X_mesh)
points = np.vstack([X_mesh.ravel(), Y_mesh.ravel(), Z_mesh.ravel()]).T
sx_pred = mx.predict(points)
SX_pred = np.reshape(sx_pred[0],X_mesh.shape)

fig = go.Figure(data=[go.Surface(x=X_mesh, y=Y_mesh, z=-SX_pred,cmin=-3,cmax=4)])
fig.add_scatter3d(x=P[select_zero_idx,0], y=P[select_zero_idx,1], z = -S[select_zero_idx,0], mode='markers', 
                  marker=dict(size=4))
fig.update_layout(#title=f'Sensor model surface plot R^2={r2_score:.3f}', 
                    title_y=0.95,
                    autosize=False,
                    scene = dict(
                        xaxis = dict(nticks=4, range=[-50,10],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,0.1)',),
                        yaxis = dict(nticks=4, range=[-50,10],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,0.1)',),
                        zaxis = dict(nticks=4, range=[-2,5],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,0.1)',),
                        xaxis_title='x axis (mm)', 
                        yaxis_title='y axis (mm)',
                        zaxis_title='z axis (mm)',),
                        #caxis = dict(range=[-10,5],),),
                    font=dict(size=16),
                    scene_camera=dict(eye=dict(x=-0.25, y=.7, z=1.5)),
                    width=900, height=700,
                    margin=dict(l=10, r=10, b=10, t=10))

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 700,
    'width': 900,
    'scale':6 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.show(config=config)



In [ ]:
import plotly.graph_objects as go
from sklearn.metrics import r2_score

# get points that are near the zero plane
select_zero_idx = np.where(np.abs(P[:,2]) < 0.001)[0]

y_pred = my.predict(P)[0]
r2_y = r2_score(y_true = S[:,1:2], y_pred = y_pred )
print(f'Sensor model R2 score {r2_y}')

margin = 20
print(min(P[:,0])-margin, max(P[:,0])+margin)
print(min(P[:,1])-margin, max(P[:,1])+margin)
x_range = np.arange(min(P[:,0])-margin, max(P[:,0])+margin)
y_range = np.arange(min(P[:,1])-margin, max(P[:,1])+margin)
[X_mesh, Y_mesh] = np.meshgrid(x_range, y_range)
Z_mesh = np.zeros_like(X_mesh)
points = np.vstack([X_mesh.ravel(), Y_mesh.ravel(), Z_mesh.ravel()]).T
sy_pred = my.predict(points)
SY_pred = np.reshape(sy_pred[0],X_mesh.shape)

fig = go.Figure(data=[go.Surface(x=X_mesh, y=Y_mesh, z=-SY_pred,cmin=-60,cmax=10)])
fig.add_scatter3d(x=P[select_zero_idx,0], y=P[select_zero_idx,1], z = -S[select_zero_idx,1], mode='markers', 
                  marker=dict(size=4))

fig.update_layout(#title=f'Sensor model surface plot R^2={r2_y:.3f}', 
                    title_y=0.95,
                    autosize=False,
                    scene = dict(
                        xaxis = dict(nticks=4, range=[-50,10],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,0.1)',),
                        yaxis = dict(nticks=4, range=[-40,10],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,0.1)',),
                        zaxis = dict(nticks=4, range=[-70,15],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,0.1)',),
                        xaxis_title='x axis (mm)', 
                        yaxis_title='y axis (mm)',
                        zaxis_title='z axis (mm)',),
                        #caxis = dict(range=[-10,5],),),
                    font=dict(size=16),
                    scene_camera=dict(eye=dict(x=-0.25, y=.7, z=1.5)),
                    width=1200, height=700,
                    margin=dict(l=10, r=10, b=10, t=10))

config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'custom_image',
    'height': 700,
    'width': 900,
    'scale':6 # Multiply title/legend/axis/canvas sizes by this factor
  }
}
fig.show(config=config)

In [ ]:
from process import ContactPointTracker
#Contact point tracker

kern = GPy.kern.ThinPlate(3, variance=1, R=100)
tracker = ContactPointTracker(fm_alpha=1.04, 
                                fm_stride=5,
                                sigma_process=1e-5, 
                                sigma_process_init=5., 
                                sigma_sensor=0.01)
sensor_atten=100
tracker.init_calibration_model(calibration_fn='../data/all_datasets/calibration/tip_calib_May_04.csv',
                               input_ref=np.array([[64.30451325,98.0687005,16.1023935]]),
                               kernel=kern, sensor_atten=sensor_atten)
tracker.initialize(track_data_fn='../data/all_datasets/test/tip_test_05_04_23/tip_test_May_04.csv',
                               input_ref=np.array([[64.30451325,98.0687005,16.1023935]]),
                                                  sensor_atten=sensor_atten)

x_ukf, sig_ukf, additional_data = tracker.run_tracker(track_mode='world')

x_est = x_ukf.T
x_err = additional_data['X_t']-x_est

selection_arr_np = np.where(np.array(additional_data['selection_arr']) == 1)[0]
fig = go.Figure()
fig.add_trace(go.Scatter(y=np.linalg.norm(x_err[selection_arr_np,:], axis=1),
                    mode='lines',
                    name='lines'))

fig.update_layout(title='Static Point Localization Error Distance', 
				xaxis_title='time', yaxis_title='distance (mm)',
				autosize=False,
				width=500, height=500,
				margin=dict(l=65, r=50, b=65, t=90))

fig.update_yaxes(range=[0,5]) 
fig.show()

# Sensor Model Straight

In [ ]:
data_tbl = pd.read_csv('../data/all_datasets/calibration/straight_calib_May_20.csv')

In [ ]:
atten = 50
pos_ref = np.array([83.57682675,95.12641312,34.03387275])

P = data_tbl[['px','py','pz']].to_numpy() - pos_ref
S = data_tbl[['sx','sy']].to_numpy()/atten

In [ ]:
#ker1 = GPy.kern.RBF(3, lengthscale=5, variance=10.0)
offset = 50

ker1 = GPy.kern.ThinPlate(3, variance=1, R=100)
#ker1 = GPy.kern.Spline(3, c=5, variance=0.5)
#ker1 = GPy.kern.Poly(3, variance=20, order=5, scale=0.05)
mx = GPy.models.GPRegression(P[offset:,:],S[offset:,0:1],ker1)
my = GPy.models.GPRegression(P[offset:,:],S[offset:,1:2],ker1)

from sklearn.metrics import r2_score

y_pred = mx.predict(P[offset:,:])[0]
r2_x = r2_score(y_true = S[offset:,0:1], y_pred = y_pred )
print(f'Sensor model X R2 score {r2_x}')
y_pred = my.predict(P[offset:,:])[0]
r2_y = r2_score(y_true = S[offset:,1:2], y_pred = y_pred )
print(f'Sensor model Y R2 score {r2_y}')

In [ ]:
import plotly.graph_objects as go
from sklearn.metrics import r2_score

# get points that are near the zero plane
select_zero_idx = np.where(np.abs(P[:,2]) < 0.001)[0]

y_pred = my.predict(P)[0]
r2_y = r2_score(y_true = S[:,1:2], y_pred = y_pred )
print(f'Sensor model R2 score {r2_y}')

margin = 20
x_range = np.arange(min(P[:,0])-margin, max(P[:,0])+margin)
y_range = np.arange(min(P[:,1])-margin, max(P[:,1])+margin)
[X_mesh, Y_mesh] = np.meshgrid(x_range, y_range)
Z_mesh = np.zeros_like(X_mesh)
points = np.vstack([X_mesh.ravel(), Y_mesh.ravel(), Z_mesh.ravel()]).T
sy_pred = my.predict(points)
SY_pred = np.reshape(sy_pred[0],X_mesh.shape)

fig = go.Figure(data=[go.Surface(x=X_mesh, y=Y_mesh, z=-SY_pred,cmin=-60,cmax=5)])
fig.add_scatter3d(x=P[select_zero_idx,0], y=P[select_zero_idx,1], z = -S[select_zero_idx,1], mode='markers', 
                  marker=dict(size=4))
fig.update_layout(title=f'Sensor model surface plot R^2={r2_y:.3f}', 
                    title_y=0.95,
                    autosize=False,
                    scene = dict(
                        xaxis = dict(nticks=4, range=[-50,10],),
                        yaxis = dict(nticks=4, range=[-40,20],),
                        zaxis = dict(nticks=4, range=[-70,5],),),
                        #caxis = dict(range=[-10,5],),),
                    width=700, height=700,
                    margin=dict(l=10, r=10, b=10, t=10))
fig.show()



# Sensor Model Curved

In [ ]:
data_tbl = pd.read_csv('../data/all_datasets/calibration/curved_calib_May_22.csv')

In [ ]:
atten = 50
pos_ref = np.array([63.479219625,98.55717525,16.102393499999998])

P = data_tbl[['px','py','pz']].to_numpy() - pos_ref
S = data_tbl[['sx','sy']].to_numpy()/atten

In [ ]:
ker1 = GPy.kern.ThinPlate(3, variance=1, R=100)
mx = GPy.models.GPRegression(P,S[:,0:1],ker1)
my = GPy.models.GPRegression(P,S[:,1:2],ker1)

from sklearn.metrics import r2_score

y_pred = mx.predict(P)[0]
r2_x = r2_score(y_true = S[:,0:1], y_pred = y_pred )
print(f'Sensor model X R2 score {r2_x}')
y_pred = my.predict(P)[0]
r2_y = r2_score(y_true = S[:,1:2], y_pred = y_pred )
print(f'Sensor model Y R2 score {r2_y}')

In [ ]:
import plotly.graph_objects as go
from sklearn.metrics import r2_score

# get points that are near the zero plane
select_zero_idx = np.where(np.abs(P[:,2]) < 0.1)[0]

y_pred = my.predict(P)[0]
r2_y = r2_score(y_true = S[:,1:2], y_pred = y_pred )
print(f'Sensor model R2 score {r2_y}')

margin = 20
x_range = np.arange(min(P[:,0])-margin, max(P[:,0])+margin)
y_range = np.arange(min(P[:,1])-margin, max(P[:,1])+margin)
[X_mesh, Y_mesh] = np.meshgrid(x_range, y_range)
Z_mesh = np.zeros_like(X_mesh)
points = np.vstack([X_mesh.ravel(), Y_mesh.ravel(), Z_mesh.ravel()]).T
sy_pred = my.predict(points)
SY_pred = np.reshape(sy_pred[0],X_mesh.shape)

fig = go.Figure(data=[go.Surface(x=X_mesh, y=Y_mesh, z=-SY_pred,cmin=-60,cmax=5)])
fig.add_scatter3d(x=P[select_zero_idx,0], y=P[select_zero_idx,1], z = -S[select_zero_idx,1], mode='markers', 
                  marker=dict(size=4))
fig.update_layout(title=f'Sensor model surface plot R^2={r2_y:.3f}', 
                    title_y=0.95,
                    autosize=False,
                    width=700, height=700,
                    margin=dict(l=10, r=10, b=10, t=10))
fig.show()



In [ ]:
import plotly.graph_objects as go
from process import ContactPointTracker
#Contact point tracker

# localization error distance with curved sensor
kern = GPy.kern.ThinPlate(3, variance=1, R=100)
tracker = ContactPointTracker(fm_alpha=1.04, 
                                fm_stride=5,
                                sigma_process=1e-5, 
                                sigma_process_init=5., 
                                sigma_sensor=0.01)
tracker.init_calibration_model(calibration_fn='../data/all_datasets/calibration/curved_calib_May_22_try2.csv',
                               input_ref=np.array([63.479219625,98.55717525,16.102393499999998]),
                               kernel=kern)
tracker.initialize(track_data_fn='../data/all_datasets/test/tip_test_05_22_23_try2/curved_test_May_22_trial9.csv',
                               input_ref=np.array([63.479219625,98.55717525,16.102393499999998]))

x_ukf, sig_ukf, additional_data = tracker.run_tracker(track_mode='world')

x_est = x_ukf.T
x_err = additional_data['X_t']-x_est

selection_arr_np = np.where(np.array(additional_data['selection_arr']) == 1)[0]
fig = go.Figure()
fig.add_trace(go.Scatter(y=np.linalg.norm(x_err[selection_arr_np,:], axis=1),
                    mode='lines',
                    name='lines'))

fig.update_layout(title='Static Point Localization Error Distance', 
				xaxis_title='time', yaxis_title='distance (mm)',
				autosize=False,
				width=500, height=500,
				margin=dict(l=65, r=50, b=65, t=90))

fig.update_yaxes(range=[0,5]) 
fig.show()

# Sensor Shape Extraction

In [ ]:
pos_ref = np.array([[64.30451325,98.0687005,16.1023935]])

data_tbl = pd.read_csv('../data/all_datasets/shape/curved_shape_x.csv')
pts_x = data_tbl[['px']].to_numpy()[:,0]
pts_z = data_tbl[['pz']].to_numpy()[:,0]

data_tbl = pd.read_csv('../data/all_datasets/shape/curved_shape_y.csv')
pts_x_new = data_tbl[['px']].to_numpy()[:,0]
pts_y = data_tbl[['py']].to_numpy()[:,0]

from scipy.interpolate import make_interp_spline, splrep, BSpline
m_shape = BSpline(*splrep(x=pts_x[::-1], y=pts_z[::-1], s=2))
X = pts_x_new - pts_x_new[0]
Y = pts_y - pts_y[0]
#Z = pts_z_interp - pts_z_interp[0]
Z = m_shape(pts_x_new) - m_shape(pts_x_new)[0]

In [ ]:

import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter3d(x=X, y=Y, z=Z,
					name='lines',mode='lines'))

camera = dict(
	up=dict(x=0, y=0, z=1),
	center=dict(x=0, y=0, z=0),
	eye=dict(x=0, y=0, z=2.)
)
fig.update_layout(title=f'Extracted Shape', 
				scene = dict(
					xaxis = dict(nticks=4, range=[-35,5],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,1)',),
					yaxis = dict(nticks=4, range=[-35,5],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,1)',),
					zaxis = dict(nticks=4, range=[-10,10],backgroundcolor='rgba(0,0,0,0)',gridcolor='rgba(0,0,0,1)',),
					aspectratio=dict(x=1, y=1, z=1)),
				xaxis_title='x axis (mm)', yaxis_title='y axis (mm)',
				autosize=False,
				width=500, height=500,
				scene_camera=camera,
				margin=dict(l=10, r=10, b=10, t=50))
fig.show()
print(X[::3])
print(X[-1])

# Save all sensor data

In [6]:
# sensor 1
# pos_ref = np.array([64.30451325,96.883680375,16.502300625])
pos_ref = np.array([64.30451325,98.0687005,16.1023935])
# load pos_ref into dataframe
pos_ref_tbl = pd.DataFrame(pos_ref.reshape(1,-1), columns=['px','py','pz'])
pos_ref_tbl.to_csv('../data/sensor_models/sensor1/sensor_pos_ref.csv', index=False)

In [15]:
import pandas as pd
sensor_num = 1
calib_date = 'May_04'
shape_date = 'May_17'
calib_file = f'../data/all_datasets/calibration/sensor{sensor_num}_tip_calib_{calib_date}.csv'
#load pos_ref from file
pos_ref = pd.read_csv(f'../data/sensor_models/sensor{sensor_num}/sensor_pos_ref.csv').to_numpy()

data_tbl = pd.read_csv(calib_file)
atten = 50
P = data_tbl[['px','py','pz']].to_numpy() - pos_ref
S = data_tbl[['sx','sy']].to_numpy()/atten
ker1 = GPy.kern.ThinPlate(3, variance=1, R=100)
mx = GPy.models.GPRegression(P,S[:,0:1],ker1)
my = GPy.models.GPRegression(P,S[:,1:2],ker1)

from sklearn.metrics import r2_score

y_pred = mx.predict(P)[0]
r2_x = r2_score(y_true = S[:,0:1], y_pred = y_pred )
print(f'Sensor model X R2 score {r2_x}')
y_pred = my.predict(P)[0]
r2_y = r2_score(y_true = S[:,1:2], y_pred = y_pred )
print(f'Sensor model Y R2 score {r2_y}')


# get sensor shape
data_tbl = pd.read_csv(f'../data/all_datasets/shape/sensor{sensor_num}_tip_shape_x_{shape_date}.csv')
pts_x = data_tbl[['px']].to_numpy()[:,0]
pts_z = data_tbl[['pz']].to_numpy()[:,0]

data_tbl = pd.read_csv(f'../data/all_datasets/shape/sensor{sensor_num}_tip_shape_y_{shape_date}.csv')
pts_x_new = data_tbl[['px']].to_numpy()[:,0]
pts_y = data_tbl[['py']].to_numpy()[:,0]

from scipy.interpolate import make_interp_spline, splrep, BSpline
m_shape = BSpline(*splrep(x=pts_x[::-1], y=pts_z[::-1], s=2))
X = pts_x_new - pts_x_new[0]
Y = pts_y - pts_y[0]
Z = m_shape(pts_x_new) - m_shape(pts_x_new)[0]

np.savez(f'../data/sensor_models/sensor{sensor_num}/sensor{sensor_num}_models.npz', P=P, S=S, model_x=mx, model_y=my, shape=np.vstack((X, Y, Z)).T)

Sensor model X R2 score 0.9999847048419511
Sensor model Y R2 score 0.9999722202698548


In [ ]:
import plotly.graph_objects as go
from sklearn.metrics import r2_score

# get points that are near the zero plane
select_zero_idx = np.where(np.abs(P[:,2]) < 0.1)[0]

margin = 20
x_range = np.arange(min(P[:,0])-margin, max(P[:,0])+margin)
y_range = np.arange(min(P[:,1])-margin, max(P[:,1])+margin)
[X_mesh, Y_mesh] = np.meshgrid(x_range, y_range)
Z_mesh = np.zeros_like(X_mesh)
points = np.vstack([X_mesh.ravel(), Y_mesh.ravel(), Z_mesh.ravel()]).T
sy_pred = my.predict(points)
SY_pred = np.reshape(sy_pred[0],X_mesh.shape)

fig = go.Figure(data=[go.Surface(x=X_mesh, y=Y_mesh, z=-SY_pred,cmin=-60,cmax=5)])
fig.add_scatter3d(x=P[select_zero_idx,0], y=P[select_zero_idx,1], z = S[select_zero_idx,1], mode='markers', 
                  marker=dict(size=4))

In [ ]:
data_dict = {
	'P': P,
    'S': S,
    'shape': np.vstack((X, Y, Z)).T,
    'name': 'sensor1'
}
np.savez_compressed('../data/sensor_models/sensor1/sensor1_models.npz', data_dict=data_dict)
data_dict_loaded=np.load('../data/sensor_models/sensor1/sensor1_models.npz', allow_pickle=True)['data_dict'].item()

ker1 = GPy.kern.Poly(3, variance=20, order=5, scale=0.05)
mx = GPy.models.GPRegression(data_dict_loaded['P'],data_dict_loaded['S'][:,0:1],ker1)
my = GPy.models.GPRegression(data_dict_loaded['P'],data_dict_loaded['S'][:,1:2],ker1)